In [1]:
import os, sys
import numpy as np
import glob
import random
import cv2

In [2]:
path_dataset = '/home/hyunjoon/dataset/lasot/crop/'
# classes = os.listdir(path_dataset)
classes = ['airplane', 'bicycle', 'boat', 'car', 'cat', 'dog', 'horse', 'motorcycle', 'train', 'truck']

In [3]:
fn_train = 'train_lasot_10class.txt'
fn_val = 'val_lasot_10class.txt'

In [4]:
total = 0
for cname in classes:
    N = len(glob.glob(os.path.join(path_dataset, cname) + '/**/*.jpg'))
    if N > 0:
        print('{}: {}'.format(N, cname))
    total += N
total

28565: airplane
13364: bicycle
16568: boat
17981: car
16005: cat
12806: dog
18284: horse
14416: motorcycle
15482: train
23924: truck


177395

In [5]:
fh_train = open(fn_train, 'w')
fh_val = open(fn_val, 'w')

for cid, cname in enumerate(classes):
    dirs = os.listdir(os.path.join(path_dataset, cname))
    train_dirs = dirs[:]
#     val_dirs = dirs[15:]

    for tdir in train_dirs:
        fn_list = glob.glob(os.path.join(path_dataset, cname, tdir) + '/*.jpg')
        fn_list = np.sort(fn_list)
        p7 = int(len(fn_list) * 0.7)
        p8 = int(len(fn_list) * 0.85)
        
        if cname in ('airplane', 'truck'):
            train_list = fn_list[:p7]
        else:
            train_list = fn_list[:p7:20]
        annot_str = '\n'.join(['{} {}'.format(fn.replace(path_dataset, ''), cid) for fn in train_list])
        fh_train.write(annot_str)
        fh_train.write('\n')
        
        val_list = fn_list[p8::10]
        annot_str = '\n'.join(['{} {}'.format(fn.replace(path_dataset, ''), cid) for fn in val_list])
        fh_val.write(annot_str)
        fh_val.write('\n')
            
#     for vdir in val_dirs:
#         fn_list = np.array(glob.glob(os.path.join(path_dataset, cname, vdir) + '/*.jpg'))
#         fn_list = np.sort(fn_list)
#         selected = np.unique(np.around(np.arange(50) / 49.0 * (len(fn_list)-1)).astype(int))
# #         random.shuffle(fn_list)
#         fn_list = fn_list[selected]
#         annot_str = '\n'.join(['{} {}'.format(fn.replace(path_dataset, ''), cid) for fn in fn_list])
#         fh_val.write(annot_str)
#         fh_val.write('\n')
            
fh_train.close()
fh_val.close()

In [6]:
train_list = open(fn_train).read().splitlines()
train_list = [os.path.join(path_dataset, l.split(' ')[0]) for l in train_list]
len(train_list)

41155

In [7]:
fn_filtered = 'train_lasot_10class_filtered.txt'
# fn_val = 'val_lasot_10class_filtered.txt'

In [8]:
def color_feature(img):
    if img.dtype == 'uint8':
        img = img.astype(float) / 255.0
    else:
        img = img.astype(float)
    r_img = cv2.resize(img, (8, 8), interpolation=cv2.INTER_AREA)
    r2 = cv2.resize(img**2, (8, 8), interpolation=cv2.INTER_AREA)
    r2 -= r_img**2
    r2 = np.sqrt(np.maximum(r2, 0))
    r_img = np.concatenate([r_img, 1.0 - r_img, r2, 1.0 - r2], axis=2).ravel()
    r_img = r_img / np.sqrt(np.sum(r_img**2))
    return r_img


def remove_duplication(fn_list):
    '''
    Args:
        fn_list: list of image filenames.

    Returns:
        filtered_list: list of filenames duplication removed.
    '''
    all_clr = [color_feature(cv2.imread(fn)) for fn in fn_list]

    N = len(fn_list)

    fidx = np.random.permutation(np.arange(N))
    # hogs = np.array(all_hog)[fidx]
    clrs = np.array(all_clr)
    kept = []
    remaining = fidx.copy()

    for ii in range(N):
        if len(remaining) == 0:
            break
        pidx = remaining[0]
        kept.append(pidx)
        if len(remaining) == 1:
            break
        
        p_clr = clrs[pidx]
        sims_clr = np.dot(clrs[remaining], p_clr) ** 100 #np.sum(clrs[remaining] * np.reshape(p_clr, (1, -1)), axis=1)**100
        
    #     p_hog = hogs[pidx]
    #     sims_hog = np.sum(hogs[remaining] * np.reshape(p_hog, (1, -1)), axis=1) / 36
        
        ridx = np.where((sims_clr) < 0.6)[0]
        remaining = remaining[ridx]
        
        if ii % 1000 == 0:
            print(ii)

    return [fn_list[idx] for idx in kept]

In [9]:
filtered_list = remove_duplication(train_list)

0
1000
2000
3000
4000
5000


In [10]:
with open(fn_filtered, 'w') as fh:
    for line in filtered_list:
        fn = line.replace(path_dataset, '')
        cid = classes.index(fn.split('/')[0])
        fh.write('{} {}\n'.format(fn, cid))

In [11]:
perclass = [0 for _ in classes]

train_list = open(fn_train).read().splitlines()
for l in train_list:
    perclass[int(l.split(' ')[1])] += 1

perclass

[19984, 476, 587, 637, 566, 457, 649, 512, 553, 16734]

In [12]:
perclass = [0 for _ in classes]

train_list = open(fn_filtered).read().splitlines()
for l in train_list:
    perclass[int(l.split(' ')[1])] += 1

perclass

[891, 412, 254, 439, 232, 318, 453, 441, 320, 1554]

In [8]:
len(train_list)

4243

In [10]:
len(filtered_list)

NameError: name 'filtered_list' is not defined